In [ ]:
from matplotlib import pyplot as plt
import matplotlib
from sklearn import preprocessing
import os
import numpy as np
import random as rd
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()
sess  = tf.compat.v1.InteractiveSession()
from scipy.integrate import odeint
import pickle
from sklearn.preprocessing import StandardScaler
from scipy.signal import savgol_filter
import joblib

In [ ]:
sess1 = tf.compat.v1.InteractiveSession()
saver = tf.compat.v1.train.import_meta_graph('Checkpoint_files/CFS_Koopman_correct_ics2.pickle.ckpt.meta', clear_devices=True)
saver.restore(sess1, tf.train.latest_checkpoint('Checkpoint_files'))

scaler = joblib.load(scaler_filename) 

psixpT = tf.compat.v1.get_collection('psiyp')[0]
psixfT = tf.compat.v1.get_collection('psiyf')[0]
xpT_feed = tf.compat.v1.get_collection('yp_feed')[0]
xfT_feed = tf.compat.v1.get_collection('yf_feed')[0]
KxT = tf.compat.v1.get_collection('Kx')[0]
KxT_num = sess1.run(KxT)


In [ ]:
N = 20
Xp_final_learned = []
Xf_final_learned = []
Xp_final_actual = []
Xf_final_actual = []
t = np.array([i for i in range(0, N+1)])
x_learned = np.zeros(2)
x_actual = np.zeros(2)
for ic in ICs:
    ic_s = scaler.transform(np.array([ic]))
    x_learned = np.array(ic_s)
    x_actual[0] = ic[0]
    x_actual[1] = ic[1]
    X_learned = []
    X_actual = []
    X_learned.append([x_learned[0][0], x_learned[0][1]]) ## ICs
    X_actual.append(np.array([x_actual[0], x_actual[1]])) 
    for k in range(0, N):
        y_learned = np.matmul(KxT_num.T, psixpT.eval(feed_dict={xpT_feed: x_learned}).T)[0:2].T
        x_learned = y_learned
        X_learned.append([x_learned[0][0], x_learned[0][1]]) 
        #print(k)
        y_actual = model(x_actual)
        x_actual = y_actual
        X_actual.append([x_actual[0], x_actual[1]])
    Xp_learned = X_learned[0:-1]
    Xf_learned = X_learned[1:]
    Xp_actual = X_actual[0:-1]
    Xf_actual = X_actual[1:]   
    X_learned = scaler.inverse_transform(X_learned)
    plt.scatter(t, np.array(X_actual).T[0])
    plt.plot(t, np.array(X_learned).T[0])
    Xp_final_learned = Xp_final_learned + Xp_learned
    Xf_final_learned = Xf_final_learned + Xf_learned
    Xp_final_actual = Xp_final_actual + Xp_actual
    Xf_final_actual = Xf_final_actual + Xf_actual
    

In [ ]:
    
N = 20
Xp_final_learned = []
Xf_final_learned = []
Xp_final_actual = []
Xf_final_actual = []
t = np.array([i for i in range(0, N+1)])
x_learned = np.zeros(2)
x_actual = np.zeros(2)
for ic in ICs:
    ic_s = scaler.transform(np.array([ic]))
    x_learned = np.array(ic_s)
    x_actual[0] = ic[0]
    x_actual[1] = ic[1]
    X_learned = []
    X_actual = []
    X_learned.append([x_learned[0][0], x_learned[0][1]]) ## ICs
    X_actual.append(np.array([x_actual[0], x_actual[1]])) 
    for k in range(0, N):
        y_learned = np.matmul(KxT_num.T, psixpT.eval(feed_dict={xpT_feed: x_learned}).T)[0:2].T
        x_learned = y_learned
        X_learned.append([x_learned[0][0], x_learned[0][1]]) 
        #print(k)
        y_actual = model(x_actual)
        x_actual = y_actual
        X_actual.append([x_actual[0], x_actual[1]])
    Xp_learned = X_learned[0:-1]
    Xf_learned = X_learned[1:]
    Xp_actual = X_actual[0:-1]
    Xf_actual = X_actual[1:]   
    X_learned = scaler.inverse_transform(X_learned)
    plt.scatter(t, np.array(X_actual).T[1])
    plt.plot(t, np.array(X_learned).T[1])
    Xp_final_learned = Xp_final_learned + Xp_learned
    Xf_final_learned = Xf_final_learned + Xf_learned
    Xp_final_actual = Xp_final_actual + Xp_actual
    Xf_final_actual = Xf_final_actual + Xf_actual


In [ ]:
   
scale = 1
if scale:
    Xp_final_learned_scaled = scaler.inverse_transform(Xp_final_learned)
    Xf_final_learned_scaled = scaler.inverse_transform(Xf_final_learned)
else:
    Xp_final_learned_scaled = np.array(Xp_final_learned)
    Xf_final_learned_scaled = np.array(Xf_final_learned)
    
#pickle.dump([Xp_ref_scaled, Xf_ref_scaled, Up_ref], open('/Users/dennisjoshy/Documents/darpa-sd2/deepDMD/koopman_data/CFS_Koopman_correct_ics2.pickle', 'wb'),  protocol = 2)

plt.scatter(np.linspace(0, len(Xp_final_learned_scaled[:, 0]), len(Xp_final_learned_scaled[:, 0])), Xp_final_learned_scaled[:, 0])
plt.scatter(np.linspace(0, len(Xf_final_learned_scaled[:, 0]), len(Xf_final_learned_scaled[:, 0])), Xf_final_learned_scaled[:, 0])

plt.scatter(np.linspace(0, len(np.array(Xp_final_actual)[:, 0]), len(np.array(Xp_final_actual)[:, 0])), np.array(Xp_final_actual)[:, 0])


In [ ]:

scale = 1
scaler = StandardScaler().fit(Xp_final_learned)
if scale:
    Xp_final_learned_scaled = scaler.inverse_transform(Xp_final_learned)
    Xf_final_learned_scaled = scaler.inverse_transform(Xf_final_learned)
else:
    Xp_final_learned_scaled = np.array(Xp_final_learned)
    Xf_final_learned_scaled = np.array(Xf_final_learned)
    
#pickle.dump([Xp_ref_scaled, Xf_ref_scaled, Up_ref], open('/Users/dennisjoshy/Documents/darpa-sd2/deepDMD/koopman_data/CFS_Koopman_correct_ics2.pickle', 'wb'),  protocol = 2)

#plt.scatter(np.linspace(0, len(Xp_final_learned_scaled[:, 1]), len(Xp_final_learned_scaled[:, 1])), Xp_final_learned_scaled[:, 1])
plt.scatter(np.linspace(0, len(Xf_final_learned_scaled[:, 1]), len(Xf_final_learned_scaled[:, 1])), Xf_final_learned_scaled[:, 1])

plt.scatter(np.linspace(0, len(np.array(Xf_final_actual)[:, 1]), len(np.array(Xf_final_actual)[:, 1])), np.array(Xf_final_actual)[:, 1])


In [ ]:
sess1.close()